### Загрузка библиотеки и подключение таблицы

In [154]:
import pandas as pd
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vTaSyz0UyaRz0NqPrDmEZHUtnlxVsyHnAC668x7Yc7ZNBwNUFqmJCReYcwlI6TIfwHMimRaqQictl7C/pub?gid=0&single=true&output=csv')

### Перевод значений в нужный тип

In [155]:
for column in ['Отправлено', 'Баунсы (Все ошибки)', 'Открытия', 'Клики', 'Отписки', 'Доставлено']:
    df[column] = df[column].str.replace('\xa0', '').astype(int)

## Расчет основных метрик и добавление их в общую таблицу

In [156]:
df['Delivery rate'] = df.apply(lambda row: round((row['Отправлено'] - row['Баунсы (Все ошибки)']) / row['Отправлено'] * 100, 2), axis=1)
df['Open rate'] = df.apply(lambda row: round((row['Открытия'] / (row['Отправлено'] - row['Баунсы (Все ошибки)']) * 100), 2), axis=1)
df['Click to Open rate'] = df.apply(lambda row: round((row['Клики'] / row['Открытия']) * 100, 2), axis=1)
df['Unsubscribe rate'] = df.apply(lambda row: round((row['Отписки'] / row['Доставлено']) * 100, 2), axis=1)

## Выявление лучшей темы

Лучшей темой считаю ту, которая: 
1. Продала наш продукт (пользователь сделал клик).
2. Меньше повлияла на количество отписок от рассылки.

In [157]:
# Вывожу 2 лучшие темы, так как их показатели одинаковые
df[['Дата', 'Тема письма ', 'Click to Open rate', 'Unsubscribe rate']].sort_values(['Click to Open rate', 'Unsubscribe rate'], ascending=[False, True]).head(2)

,Дата,Тема письма,Click to Open rate,Unsubscribe rate
144,13.01.2022,Тема письма 211,12.0,0.15
158,19.01.2022,Тема письма 216,12.0,0.15


## Выявление лучшего дня недели (для планирования рассылок)

Лучшим днем считаю тот, в который: 
1. Больше пользователей ознакомились с продуктом/темой (открыли письма).
2. Больше пользователей купили продукт (сделали клик).
3. Меньше пользователей отписалось от рассылки.

### Создание таблицы с показателями по дням недели

In [158]:
mail_by_dayweek = df.groupby('День недели.1', as_index = False).agg({'Отправлено' : 'sum', 'Баунсы (Все ошибки)' : 'sum', 'Открытия' : 'sum', 'Клики' : 'sum', 'Отписки' : 'sum', 'Доставлено' : 'sum'})

### Расчет нужных метрик и добавление их в таблицу

In [159]:
mail_by_dayweek['Open rate'] = mail_by_dayweek.apply(lambda row: round((row['Открытия'] / (row['Отправлено'] - row['Баунсы (Все ошибки)']) * 100), 2), axis=1)
mail_by_dayweek['Click to Open rate'] = mail_by_dayweek.apply(lambda row: round((row['Клики'] / row['Открытия']) * 100, 2), axis=1)
mail_by_dayweek['Unsubscribe rate'] = mail_by_dayweek.apply(lambda row: round((row['Отписки'] / row['Доставлено']) * 100, 2), axis=1)

### Сортировка и вывод

In [160]:
mail_by_dayweek[['День недели.1', 'Open rate', 'Click to Open rate', 'Unsubscribe rate']].sort_values(['Open rate', 'Click to Open rate', 'Unsubscribe rate'], ascending=[False, False, True]).head(1)

,День недели.1,Open rate,Click to Open rate,Unsubscribe rate
2,03-среда,14.28,8.39,1.25
